# __From Narrow AI to Generative AI: Enhancing your DFIR Skills with AI__
__SANS-AI Cybersecurity Summit__

*by Jess Garcia (@j3ssgarcia) and Mario Perez*

## 🌙🚨 __The Midnight Intruder Case__ 🌙🚨

In the dead of night 🦉, an intruder gained unauthorized access to the company's network via a __compromised user account obtained through phishing__ 🎣. Their goal was to deploy ransomware 💻🔒, but to do so, they needed administrative privileges. After several hours of __brute-forcing__ 🔑💥, they succeeded in cracking an admin password, __escalating privileges__ 🚀. However, the __creation of an admin domain controller account 'sql001'__ 🖥️ triggered an alert 🚨, allowing the IT team to detect the breach just minutes before the ransomware could be deployed.

Can you trace the intruder's steps, identify the breached accounts and machines 🕵️‍♂️ to neutralize the threat before it's too late? ⏳

__For your referece, this is the Attackmap of the Threat Actor__

_https://github.com/ds4n6/sans_narrow-ai_notebooks/blob/main/attackmap.jpeg_

<div>
<img src="attackmap.jpeg" width="890"/>
</div>

<span style="color:RED">**In the following links you can find the source Jupyter Notebooks for reference:**</span>
* Notebook: https://github.com/ds4n6/sans_narrow-ai_notebooks/blob/main/SANS_TheMidnightIntruderCase.ipynb 
* __SOLVED__ Notebook: https://github.com/ds4n6/sans_narrow-ai_notebooks/blob/main/SOLVED_TheMidnightIntruderCase.ipynb

### __FORENSIC EVIDENCE__ 

This section describes the evidence we will be examining in this case. Our focus will be on analyzing Windows EVTX files. 

__Windows EVTX files__ are log files utilized by the Windows operating system to document various events, including system, security, and application activities. These logs can capture events ranging from software installations to security incidents and system errors. The EVTX format is binary, which offers advantages in terms of storage efficiency and retrieval speed compared to plain text logs. __To analyze these files, we can use tools such as the Windows Event Viewer, or specialized parsers like EvtxECmd or Sabonis__.

#### __Sabonis__ 

Sabonis is a __Digital Forensics and Incident Response pivoting tool__ for parsing different kind of forensic artifacts realted to lateral movements like __EVTX, proxy and PCAP files__. These are the main features of the tool:
* 🔍 Extracts and merge lateral movements from more than 7 different EVTX files
* 🔍 Parses Squid proxy events
* 🔍 Extracts all lateral movements from PCAP files
* ⚡ Quick and low memory comsumption
* 📑 Loads different sources into a Neo4J database
* 🔍 Includes a Cypher Playbook to make investigations easy

You can find more information at: https://github.com/jupyterj0nes/sabonis

#### __Creating a Lateral Movement Dataset with Sabonis__

Sabonis is a powerful tool for __consolidating lateral movement event logs from your network into a single CSV dataset__ for analysis. It processes lateral movement events from EVTX files, including:

__C:/Windows/System32/winevt/Logs/__
* _Security.evtx_
* _Microsoft-Windows-SMBServer%4Security.evtx_
* _Microsoft-Windows-SmbClient%4Security.evtx_
* _Microsoft-Windows-TerminalServices-RDPClient%4Operational.evtx_
* _Microsoft-Windows-RemoteDesktopServices-RdpCoreTS%4Operational.evtx_

__Using Sabonis__

In [15]:
# Do not run this cell; it is just an example Sabonis' usage. It is meant to be run in the console. Pre-cooked evidence will be provided to you later.

# ./pivotfoot.sh source_folder_with_evtxs/ destination_folder/
# python3 sabonis.py parse evtx --source_artifact destination_folder/ --directory --csv_output sabonis_output.csv

https://github.com/jupyterj0nes/sabonis?tab=readme-ov-file#examples

__Pre-cooked Evidence__

In this lab, you'll be working with pre-cooked evidence generated by running Sabonis on various EVTX files collected from different machines on the network, including domain controllers, servers, and workstations.

You can find the consolidated file, __sabonis_output.csv__, in the /datasets folder within the same directory as this notebook: https://github.com/ds4n6/sans_narrow-ai_notebooks.

## __LAB-001:__ Anomaly Detection with ML for DFIR

With all the necessary information in hand, we can start the investigation 🕵️‍♂️.

__Our goal__ is to monitor the intruder's activities, evaluate the nature of the attack, and assess the extent of the damage caused. To efficiently achieve this, we will leverage advanced __Data Science and Machine Learning__ techniques. This approach will enable us to analyze patterns, detect anomalies, and gain insights that are crucial for a comprehensive understanding of the intrusion. Let’s get started!!

To carry out the analysis, we will use the __ds4n6_lib__ python library, also known as __CHRYSALIS__ (https://github.com/ds4n6/ds4n6_lib).

__CHRYSALIS__ is a Python library for __Digital Forensics and Incident Response (DFIR)__ that enables the application of __AI/ML analyses__ on critical forensic artifacts like EVTX files, Amcache, Autoruns, or Pslist. It can also work directly with the output from various forensic tools, including Plaso, Kape, Fstl, Kansa, Mactime, Volatility, or Sabonis, among others.

__To start the analysis__, we first need to load the Sabonis module from CHYRSALIS, which allows us to read and work with Sabonis datasets. Then, we’ll load the "_explore_" functionality, which we’ll use later to examine the dataset's content and conduct a preliminary analysis.

In [8]:
# IMPORT LIBRARIES
import ds4n6_lib.sabonis as d4sbns
from ds4n6_lib.tools import explore

#### __TASK 001-001:__ Read the Sabonis Dataset with CHRYSALIS

__Run the following cell__ to read the Sabonis dataset and displays it on the screen. The dataset is loaded as a __Pandas dataframe__ and stored in the variable '__sbnsdf__'.

__NOTE:__ A Pandas DataFrame is a python data structure used for storing datasets. It organizes data into rows and columns, making it straightforward to manipulate, analyze, and visualize the data. The Pandas library also offers a range of powerful functions to handle, clean, and analyze data within these DataFrames.

In [10]:
path = './datasets/sabonis_output.csv'
sbnsdf = d4sbns.read_data(path, tool='sabonis')
sbnsdf

- Saved pickle file found:
      ./datasets/sabonis_output.csv.ham.pkl
- Reading data from pickle file.
- Done.


,D4_DataType_,D4_Orchestrator_,D4_Tool_,D4_Plugin_,D4_Hostname_,Timestamp_,Computer_,EventID_,ComputerName_,ComputerDomain_,...,UserID_,TargetUserName_,TargetComputer_,LogonType_,SourceComputer_,SourceIP_,evtxFileName_,TargetHostname_,TargetDomain_,TargetIP_
0,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.543439,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
1,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.543439,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
2,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.590322,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
3,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.590322,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
4,sabonis,NaN,sabonis,NaN,NaN,2017-12-04 13:04:25.024486,dc-core10.corpnet1.local,31001,dc-core10,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83285,sabonis,NaN,sabonis,NaN,NaN,2023-12-15 13:05:00.000816,svr-file01.corpnet2.local,4625,svr-file01,corpnet2.local,...,svr-file01$,administrator,wrkstn-pro01,4,svr-file01,-,/windows/system32/winevt/logs/security.evtx,wrkstn-pro01,d4_null,d4_null
83286,sabonis,NaN,sabonis,NaN,NaN,2023-12-16 04:22:51.548426,wrkstn-pro03.corpnet2.local,4624,wrkstn-pro03,corpnet2.local,...,wrkstn-pro03$,system,nt authority,5,-,-,/windows/system32/winevt/logs/security.evtx,nt authority,d4_null,d4_null
83287,sabonis,NaN,sabonis,NaN,NaN,2023-12-16 05:13:11.923656,wrkstn-pro03.corpnet2.local,4624,wrkstn-pro03,corpnet2.local,...,wrkstn-pro03$,system,nt authority,5,-,-,/windows/system32/winevt/logs/security.evtx,nt authority,d4_null,d4_null
83288,sabonis,NaN,sabonis,NaN,NaN,2023-12-16 05:22:51.545827,wrkstn-pro03.corpnet2.local,4624,wrkstn-pro03,corpnet2.local,...,wrkstn-pro03$,system,nt authority,5,-,-,/windows/system32/winevt/logs/security.evtx,nt authority,d4_null,d4_null


__Understanding the Dataset__

As previously mentioned, Sabonis parses data from various EVTX files to extract information related to lateral movements. Some of the key fields of interest include:
* **Timestamp_**: The date and time of the event.
* **EventID_**: The Windows event identifier.
* **LogonType_**: The type of logon (Interactive, Network, Batch, Remote-Interactive...).
* **ComputerName_**: The target machine of the lateral movement (destination machine).
* **TargetUserName_**: The name of the user that logons on the target machine.
* **SourceComputer_**: The machine from which the lateral movement originated (source machine).
* **SourceIP_**: The IP address of the source machine.
* **evtxFileName_**: The name of the EVTX file containing the event data.

💡 **Remember**: Windows Logon Types and Logon Codes  💡
* Interactive (Logon Type 2)
* Network (Logon Type 3)
* Batch (Logon Type 4)
* Service (Logon Type 5)
* Unlock (Logon Type 7)
* NewCredentials (Logon Type 9)
* RemoteInteractive (Logon Type 10)

_https://www.manageengine.com/products/active-directory-audit/learn/what-are-logon-types.html_

Take some time to review the dataset and understand the data it contains. Use the down bar to move to the right or to the left. __As you can see, the dataset has more than 83k events!!!__

#### __TASK 001-002:__ Explore the Dataset

Before diving into more complex analysis, we'll begin with a __preliminary exploration__ to assess the dataset's scope, the types of data it contains, and key metrics such as the number of machines or users contained in the dataset. 

An __initial qualitative analysis__ is usually __quick__, aids in gaining a __clearer understanding__ of the dataset, and often __helps in identifying anomalies early on__.

To explore the dataset we are going to use the __CHRYSALIS__ function __'explore()'__ that we loaded at the beginning. This function takes an input DataFrame (df) and a column name (col) as arguments, and __prints the counts of unique values__ in that column of the dataset. 

__Run the following cell__ to analyze the users involved in lateral movements within the network. We will focus on the '_TargetUserName__' column of the dataset '__sbnsdf__' for this analysis.

In [12]:
explore(sbnsdf,"TargetUserName_")

#Count: 43
TargetUserName_
administrator                                      32639
d4_null                                            29288
anonymous logon                                     7017
default_value                                       4803
admin                                               3059
nt authority\anonymous logon                        2402
system                                              1033
dc-core01$                                           779
j693                                                 696
cloud                                                592
localroot                                            211
it-support                                           203
adminsys                                             192
s-1-5-21-2557963547-1975000157-7004567511-500         43
user                                                  40
s-1-5-21-2466637891-1508815653-3312488790-500         35
dwm-1                                                 30
admi

As you can see, the user '__administrator__' appears in 32,639 events as the target username, along with other users such as '__admin__', '__localroot__', or '__it-support__'.

__Now, use the CHRYSALIS' explore() funtion to answer the following questions...__

<span style="color:blue">**What is the most repeated event IDs in the dataset?** </span>

In [14]:
# Put your code here and run the cell (TIP: check again the name of the columns of the dataset)
explore(sbnsdf,"EventID_")

#Count: 17
EventID_
551      35587
4625     34283
31001     4803
1009      2419
4624      1519
24        1007
25         970
4648       730
131        689
1149       527
4634       426
21          82
22          82
1024        61
1102        56
4776        47
4647         2
Name: count, dtype: Int64


<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">__Event ID 551__ appears 35,587 times in the dataset. This event is related to __SMB Session Authentication Error__. For now, we'll exclude these events from our analysis because a closer examination reveals that many of them are generated by system processes.</span>
* <span style="color:green">__Event ID 4625__ appears 34,283 times in the dataset. This event is more familiar to us as it corresponds to __Failed Login Attempts__. These events are generated whenever a user attempts to log in to a system but the authentication fails due to several reasons like incorrect username or password. We are concerned by the high number of failed login attempts compared to the number of successful logons (__Event ID 4624__).</span>

<span style="color:blue">**What is the machine that receives the most traffic on the network?** </span>

In [16]:
# Put your code here and run the cell
explore(sbnsdf,"ComputerName_")

#Count: 27
ComputerName_
wrkstn-100      44449
dc-core01       14707
svr-file01       6516
wrkstn-dev02     5256
wrkstn-pro02     5166
dc-core02        2453
wrkstn-pro04     1183
wrkstn-pro03      987
wrkstn-pro01      479
dc-core03         304
svr-bkup04        251
dc-core04         204
svr-bkup03        197
svr-bkup05        194
svr-bkup01        193
svr-bkup06        180
dc-core08         170
svr-bkup02        131
d4_null           117
dc-core06          99
tempwindows        12
wrkstn-002         12
svr-bkup08         10
corpnet2            7
wrkstn-dev01        6
wrkstn-004          4
dc-core10           3
Name: count, dtype: Int64


<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">The machine __WRKSTN-100__ receives most of the traffic of the network (__44,449 events__), approximately __x4 times more events that the main domain controller__ of the network (14,707 events). It is interesting to note that this workstation is receiving much more traffic than any domain controller or server on the network. This activity raises some concerns, so we will keep an eye on this workstation.</span>

#### __TASK 001-003:__ Machine Learning Analysis

__Now's when the magic happens! ⭐__

__CHRYSALIS__ is designed for analyzing diverse forensic artifacts by using __Artificial Intelligence and Machine Learning__ techniques. The library implements high-level functions to assist forensic experts in their investigations. When analyzing __Sabonis datasets__, CHRYSALIS organizes logon events into two categories: successful (__event ID 4624__) and failed (__event ID 4625__), grouping them by minutes. Whith these processed samples, CHRYSALIS trains Machine Learning models to __identify the most suspicious events on the network__.

__Run the following cell__ to apply ML techniques for detecting the most suspicious login activity on the network. For this analysis, we just need to run the CHRYSALIS' function __ml_logons_anomalies()__. This funtion receives three arguments:
* __df__ = the Sabonis dataset to analyze, in our case '__sbnsdf__'
* __model__ = the ML model used for anomaly detection. Here, we use the model __iForest (Isolation Forest)__, a simple yet effective ML method for identifying outliers in large datasets (_https://en.wikipedia.org/wiki/Isolation_forest_).
* __top_n__ = the number of anomalies or outliers to display.

The funtion __returns__ a sorted list with the most anomalous samples of the dataset. Note that samples at the beginning of the list will be more anomalous than those toward the end. 

__TIP__: To analyze the results, focus on the columns "**EventID_4624_**" and "**EventID_4625_**". They show the __count of events 4624 and 4625__ where the user '**TargetUserName_**' tried to access to the target machine '**LatMov_**', within the minute specified in the column '**Timestamp_**'.

In [18]:
logon_anomalies_iforest = d4sbns.ml_logons_anomalies(sbnsdf, model='iforest', top_n=50)
logon_anomalies_iforest

,Timestamp_,LatMov_,TargetUserName_,LogonType_,EventID_4624_,EventID_4625_,score
0,2023-12-11 19:24:00,[172.17.10.253]->[wrkstn-100],admin,3,0,1413,-0.251560
1,2023-12-11 19:25:00,[172.17.10.253]->[wrkstn-100],admin,3,0,1202,-0.248577
2,2023-12-11 19:09:00,[172.17.10.253]->[wrkstn-100],administrator,3,1,1433,-0.245846
3,2023-12-11 19:22:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1493,-0.244161
4,2023-12-11 19:10:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1458,-0.243826
5,2023-12-11 19:17:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1442,-0.243539
6,2023-12-11 19:11:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1446,-0.243539
7,2023-12-11 19:15:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1438,-0.243491
8,2023-12-11 19:05:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1434,-0.243395
9,2023-12-11 19:14:00,[172.17.10.253]->[wrkstn-100],administrator,3,0,1421,-0.243098


<span style="color:blue">**Identify two possible users who may have been abused with brute force** </span>

<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">The users '__admin__' and '__administrator__' might have been targeted by a brute force attack, as evidenced by the unusually high number of failed login attempts over a short period. The table reveals that these users have more than 1,000 failed logins per minute.</span>

<span style="color:blue">**What day and time does the brute force campaign take place?** </span>

<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">The brute force campaign occurs on __2023-12-11 at 7pm__</span>

<span style="color:blue">**Based on the results, Do you think the brute force campaign was successful?** </span>

<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">Yes, it appears that the brute force attack was successful. If we look at __timestamp 2023-12-11 19:09:00__, among over __1,000 failed login attempts__, the user '__administrator__' successfully logged into '__WRKSTN-100__'. This event was the first successful lateral movement of the threat actor after compromising the machine with IP '__172.17.10.253__'.</span>

__Autoencoders for Anomaly Detection__

In CHRYSALIS, we can use either __iForest__ models or __Deep Learning Autoencoders__ for detecting anomalies in our datasets. To accomplish this, simple switch the model to '__simple_autoencoder__' when using the __ml_logons_anomalies()__ function. This allows you to easily switch between different anomaly detection techniques based on your specific needs and dataset characteristics.

💡 **What is an Autoencoder?** _https://www.kaggle.com/code/robinteuwens/anomaly-detection-with-auto-encoders_

Note that we can set the number of training epochs for the ML model using the '__train_epochs__' argument. We'll keep the number of epochs low to prevent overtraining the network, improving the capabilities of the model to detect anomalies.

In [20]:
logon_anomalies_ae = d4sbns.ml_logons_anomalies(sbnsdf, model='simple_autoencoder', top_n=50, train_epochs=3)
logon_anomalies_ae

Training Autoencoder...
_____________________________________________________________________________________
Epoch 1/3
552/552 [==============================] - 1s 836us/step - loss: 0.0194 - mse: 0.0322
Epoch 2/3
552/552 [==============================] - 1s 929us/step - loss: 0.0021 - mse: 0.0043
Epoch 3/3
173/173 [==============================] - 0s 524us/step


,Timestamp_,LatMov_,TargetUserName_,LogonType_,EventID_4624_,EventID_4625_,score
0,2023-12-14 08:28:00,[-]->[wrkstn-pro04],dwm-1,2,1,0,0.052061
1,2023-12-12 16:32:00,[172.17.10.246]->[dc-core01],sql001,10,2,0,0.050130
2,2023-12-14 08:28:00,[-]->[wrkstn-pro04],localroot,2,1,0,0.047152
3,2023-12-11 19:24:00,[172.17.10.253]->[wrkstn-100],admin,3,0,1413,0.041290
4,2023-12-11 19:25:00,[172.17.10.253]->[wrkstn-100],admin,3,0,1202,0.039614
5,2023-12-13 01:33:00,[-]->[wrkstn-100],administrator,2,1,0,0.039091
6,2023-12-13 00:52:00,[-]->[wrkstn-100],administrator,2,1,0,0.039091
7,2023-12-13 04:29:00,[-]->[wrkstn-100],administrator,2,1,0,0.039091
8,2023-12-13 05:12:00,[-]->[wrkstn-100],administrator,2,1,0,0.039091
9,2023-12-15 12:00:00,[-]->[dc-core01],dwm-1,2,4,0,0.038182


Analyzing the results, we note that __the Autoencoder also highlights some events indicative of brute force attacks__. However, the __iForest model demonstrated superior performance in detecting brute force__ attempts made by the user '__administrator__' on the target machine '__WRKSTN-100__'.

<span style="color:blue">**[Advanced] On the Autoencoder output, Do you see any signs of suspicious activity that the iForest model did not detect?** </span>

__TIP: Pay special attention to the information revealed by the name of the machines and the type of traffic they typically receive.__

<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">Examining the event related to the user '__sql001__', it shows a lateral movement from the workstation '__WRKSTN-100__' (__172.17.10.246__) to the domain controller '__DC-CORE01__', with a logon event of type 10. __Event Logon Type 10__ refers to a "Remote Interactive Logon," which occurs when a user accesses a computer remotely via Remote Desktop Protocol (RDP) or similar services. It is rare to find Logon Type 10 events on domain controllers because direct RDP connections to these critical servers are typically restricted for security reasons. Instead, administrators usually manage domain controllers using other methods that do not require Remote Desktop, such as using Remote Server Administration Tools.</span>
* <span style="color:green">Upon reviewing the remaining events and forensic artifacts, it was found that '__sql001__' was a user account created by the threat actor with __domain admin privileges__. The threat actor intended to use this account to deploy ransomware from the domain controllers.</span>

## __LAB-002:__ Graphs & ML for Lateral Movement Detection (OPTIONAL)

We know that __the threat actor has gained access to multiple machines__ on the network and compromised two user accounts: '__administrator__' and '__sql001__'. In this lab, we'll conduct a more __in-depth analysis of the lateral movements__ within the network.

__Graph-based analysis__ is a highly effective method for examining network graphs and identifying suspicious lateral movements (_Graphs for DFIR Analysis. The Roadmap: http://www.ds4n6.io/blog/23050801.html_). However, in large-scale networks, this approach can become complicated due to the vast number of events occurring within the network. In these scenarios, the malicious activity of an attacker may be masked by the background noise generated by legitimate users, systems, or IT administrators.

__CHRYSALIS for Graph Analysis__

Since the release 0.8.1, the __ds4n6_lib__ library implements a module to __apply Machine Learning on Graphs__ to discover __stealth actors moving laterally through large networks__. With the module __mlgraph__, we can represent the logon activity of a network in a graph and __apply advanced Machine Learning models__ to detect suspicious lateral movements in the network.

To begin with the __lateral movement analysis__, we first need to load the __mlgraph__ module of CHRYSALIS. Next, we will reload the Sabonis dataset as we did in the previous lab session.

In [21]:
import ds4n6_lib.mlgraph as d4mlg
import ds4n6_lib.sabonis as d4sbns

path = './datasets/sabonis_output.csv'
sbnsdf = d4sbns.read_data(path, tool='sabonis')
sbnsdf

- Saved pickle file found:
      ./datasets/sabonis_output.csv.ham.pkl
- Reading data from pickle file.
- Done.


,D4_DataType_,D4_Orchestrator_,D4_Tool_,D4_Plugin_,D4_Hostname_,Timestamp_,Computer_,EventID_,ComputerName_,ComputerDomain_,...,UserID_,TargetUserName_,TargetComputer_,LogonType_,SourceComputer_,SourceIP_,evtxFileName_,TargetHostname_,TargetDomain_,TargetIP_
0,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.543439,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
1,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.543439,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
2,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.590322,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
3,sabonis,NaN,sabonis,NaN,NaN,2017-10-27 03:01:08.590322,svr-bkup08.corpnet1.local,31001,svr-bkup08,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
4,sabonis,NaN,sabonis,NaN,NaN,2017-12-04 13:04:25.024486,dc-core10.corpnet1.local,31001,dc-core10,corpnet1.local,...,d4_null,default_value,d4_null,3,wrkstn-pro01.corpnet1.local,WRKSTN-PRO01.corpnet1.local,/windows/system32/winevt/logs/microsoft-window...,d4_null,d4_null,d4_null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83285,sabonis,NaN,sabonis,NaN,NaN,2023-12-15 13:05:00.000816,svr-file01.corpnet2.local,4625,svr-file01,corpnet2.local,...,svr-file01$,administrator,wrkstn-pro01,4,svr-file01,-,/windows/system32/winevt/logs/security.evtx,wrkstn-pro01,d4_null,d4_null
83286,sabonis,NaN,sabonis,NaN,NaN,2023-12-16 04:22:51.548426,wrkstn-pro03.corpnet2.local,4624,wrkstn-pro03,corpnet2.local,...,wrkstn-pro03$,system,nt authority,5,-,-,/windows/system32/winevt/logs/security.evtx,nt authority,d4_null,d4_null
83287,sabonis,NaN,sabonis,NaN,NaN,2023-12-16 05:13:11.923656,wrkstn-pro03.corpnet2.local,4624,wrkstn-pro03,corpnet2.local,...,wrkstn-pro03$,system,nt authority,5,-,-,/windows/system32/winevt/logs/security.evtx,nt authority,d4_null,d4_null
83288,sabonis,NaN,sabonis,NaN,NaN,2023-12-16 05:22:51.545827,wrkstn-pro03.corpnet2.local,4624,wrkstn-pro03,corpnet2.local,...,wrkstn-pro03$,system,nt authority,5,-,-,/windows/system32/winevt/logs/security.evtx,nt authority,d4_null,d4_null


#### __TASK 002-001:__ Lateral Movement Dataset

__Thinking in Graphs__

For thinking in graphs, we first need to represent our data in that format. So, the initial step is to convert the data into a graph representation. CHRYSALIS makes this easier with the function '__build_lm_dataset()__'. This function turns an event log dataset into a graph by creating two matrices: the feature matrix and the adjacency matrix. It then tie the user sessions based on the logon events timestamps. Finally, it generates a new dataset containing details about user activity sub-graphs, which represents the __users' lateral movements__ in the network. 

More details about '__build_lm_dataset()__' funtion at http://www.ds4n6.io/blog/23091501.html

__Run the following cell__ to tie the events logs and generate a __new dataset with the complete lateral movements__ of the users in the network. As argument, '__build_lm_dataset()__' receives the Sabonis dataset we loaded before.

In [22]:
lm_dataset = d4mlg.build_lm_dataset(sbnsdf)
lm_dataset

,date,user,path,timestamp
0,2021-08-24,administrator,"[172.16.110.75, wrkstn-dev02]",[00:51:21]
1,2021-08-25,administrator,"[172.16.110.31, wrkstn-dev02]",[07:03:20]
2,2021-08-25,adminsys,"[172.17.19.81, wrkstn-dev02]",[08:24:42]
3,2021-08-26,adminsys,"[172.17.19.93, wrkstn-dev02]",[03:22:54]
4,2021-08-26,administrator,"[172.16.110.124, wrkstn-dev02]",[08:26:38]
...,...,...,...,...
645,2023-12-13,administrator,"[win10-int, wrkstn-100, 192.168.10.4]","[00:23:58, 00:34:23]"
646,2023-12-13,administrator,"[win10-int, wrkstn-100, 192.168.10.5]","[00:23:58, 00:36:28]"
647,2023-12-13,administrator,"[win10-int, wrkstn-100, 192.168.10.96]","[00:23:58, 00:40:02]"
648,2023-12-13,sql001,"[192.168.10.1, wrkstn-pro04]",[00:17:02]


As shown, __CHRYSALIS has identified 650 lateral movements__ from our dataset. Initially, we observe that these movements vary in length, with some being 2 jumps long and others extending to 3 jumps. The '__path__' column displays the sequence of movements made by a user within a specific time period.

#### __TASK 002-002:__ Machine Learning Analysis

After building the new dataset with the lateral movements of the network, let's analyze it for searching anomalies. For this purpose, CHRYSALIS provides the '__find_lm_anomalies()__' function. This function leverages an __Autoencoder based on Transformers__ to __identify the most suspicious lateral movements in the network__.

__find_lm_anomalies()__ function requires six parameters:
* __dataset__: The lateral movements dataset generated CHRYSALIS.
* __model__: The type of ML model for anomaly detection.
* __from_date__: The start date of the analysis.
* __to_date__: The end date of the analysis.
* __top_n__: The number of anomalies or outliers to display.
* __neo4j__: Indicates whether to generate a compatible Neo4j dataset to load the results into a Neo4j database for further analysis.

Further information here: http://www.ds4n6.io/blog/23091501.html

__Run the following cell__ to look for suspicious lateral movements. Initially, you will observe the Autoencoder training process. Afterward, the model will show a ranking of the most anomalous lateral movements in the network. 

__NOTE__: The '_Error_' parameter indicates the anomaly level, where 100% signifies a highly anomalous movement and 0% indicates minimal anomaly. Keep in mind that most machine learning models include a stochastic component, meaning that running the model multiple times can yield different outcomes. Nonetheless, if you run the model several times, you'll notice that the results tend to be quite similar.

In [24]:
output = d4mlg.find_lm_anomalies(lm_dataset, model='transformer', from_date='2023-01-01', to_date='2024-01-17',top_n=30, neo4j=False)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 positional_embedding_2 (Po  (None, None, 16)             1504      ['encoder_inputs[0][0]']      
 sitionalEmbedding)                                                                               
                                                                                                  
 decoder_inputs (InputLayer  [(None, None)]               0         []                            
 )                                                                                      

<span style="color:blue">**Lists the lateral movements of the threat actor detected by the model (As reference, use the attack map provided at the beginning of the notebook)** </span>

__Note: Results may vary due to the stochastic nature of the model. We suggest running the model multiple times and reviewing the outcomes.__

<span style="color:green">**ANSWER HERE**:</span>
* <span style="color:green">On 2023-12-11 the user '__administrator__' moved from:</span>
    * <span style="color:green">__wrkstn-013__ -[to]-> __wrkstn-100__</span>
* <span style="color:green">On 2023-12-12 the user __administrator__ moved from:</span>
    * <span style="color:green">__wrkstn-013__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.1__</span>
* <span style="color:green">On 2023-12-12 the user __administrator__ moved from:</span>
    * <span style="color:green">__10.6.125.39__ -[to]-> __dc-core01__</span>
* <span style="color:green">On 2023-12-12 the user __sql001__ moved from:</span>
    * <span style="color:green">__192.168.10.1__ -[to]-> __wrkstn-pro03__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__wrkstn-013__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.2__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__wrkstn-013__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.4__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__wrkstn-013__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.5__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__wrkstn-013__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.96__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__win10-int__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.2__</span>
* <span style="color:green">One 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__win10-int__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.4__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__win10-int__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.5__</span>
* <span style="color:green">On 2023-12-13 the user __administrator__ moved from:</span>
    * <span style="color:green">__win10-int__ -[to]-> __wrkstn-100__ -[to]-> __192.168.10.96__</span>
* <span style="color:green">On 2023-12-13 the user __sql001__ moved from:</span>
    * <span style="color:green">__192.168.10.1__ -[to]-> __wrkstn-pro04__</span>
* <span style="color:green">On 2023-12-13 the user __sql001__ moved from:</span>
    * <span style="color:green">__172.17.10.246__ -[to]-> __dc-core01__</span>

__Congratulations on completing the cybersecurity incident response challenge!!__ 🎉

You’ve successfully navigated through this real-world CTF scenario, demonstrating your skills and creativity in __solving complex problems with AI and Machine Learning__.

If you enjoyed this challenge and are eager to explore more content like this, we invite you to visit our website at https://www.ds4n6.io/. There, you'll find a variety of engaging and educational resources designed to further enhance your __Threat Detection & Response__ expertise.

Thank you for participating, and we look forward to seeing you in future challenges! 🚀